In [1]:
import pandas as pd
import numpy as np
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
import warnings

warnings.simplefilter('ignore')
np.random.seed(42)

## Preprocessing

In [12]:
df = pd.read_csv("data/mood_aggregated.csv")

# We only care about mood for ARIMA, so we drop other variables
df = df[df.variable == 'mood']

# We keep the id, timestamp, and value for mood
df = df[['id', 'time', 'value']]

# We change the time strings to datetime objjects
df.time = pd.to_datetime(df.time)
df.time = df.time.dt.floor('H')

# We rename the value column to mood since it only contains values for mood now
df = df.rename(columns={'value': 'mood'})

df = df.dropna()

## Separating the data
* First we will separate for each participant
* Then we will do a train/test split for each

In [3]:
# We split the data for each participant
participants = {participant: df[df.id == participant].drop(['id'], axis=1) for participant in df.id.unique()}

# Maps each participants to a train and test set
train_test_dict = {}
for participant in participants:
    train = list(participants[participant]['mood'])[:-1]
    test = list(participants[participant]['mood'])[-1]
    
    train_test_dict[participant] = {'train': train, 'test': test}


## Finding the models with the optimal parameters

In [4]:
# Maps each participant to a fitted model
model_dict = {}
for participant in participants:
    train = train_test_dict[participant]['train']
    
    model = pm.auto_arima(train, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=5, max_q=5, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality (Might have to check for this shit?)
                          start_P=0, 
                          D=0, 
                          trace=False,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
    model_dict[participant] = model

## Using the statsmodels package ARIMA function

In [5]:
final_models = {}
fitted_models = {}
for participant in model_dict.keys():
    order = model_dict[participant].order
    endog = train_test_dict[participant]['train']
    model = ARIMA(endog=endog, order=order)
    
    final_models[participant] = model
    
    fitted_models[participant] = model.fit()

## Predicting with each model

In [6]:
prediction_dict = {}
for participant in participants:
    # Get the correct model
    model = fitted_models[participant]
    
    # Get the real labels and cutoff
    true_labels = [train_test_dict[participant]['test']]
    
    
    size = len(train_test_dict[participant]['train'])
    
    # Predict
    predictions = model.predict(start=size, end=size)
    
    assert len(predictions) == len(true_labels)
    
    # Save the predictions
    prediction_dict[participant] = {'true_labels': true_labels, 'predictions': predictions}

In [7]:
for participant in prediction_dict:
    true_value = prediction_dict[participant]['true_labels']
    prediction = prediction_dict[participant]['predictions']
    
    MAE = mean_absolute_error(true_value, prediction)
    MSE = mean_squared_error(true_value, prediction)
    
    print(f"{participant}     MAE = {MAE:.2f}     MSE = {MSE:.2f}     True_Value = {true_value[0]:.2f}       Predicted = {list(prediction)[0]:.2f}")

AS14.01     MAE = 0.22     MSE = 0.05     True_Value = 8.00       Predicted = 7.78
AS14.02     MAE = 2.89     MSE = 8.32     True_Value = 9.00       Predicted = 6.11
AS14.03     MAE = 0.29     MSE = 0.08     True_Value = 7.67       Predicted = 7.38
AS14.05     MAE = 0.58     MSE = 0.34     True_Value = 6.33       Predicted = 6.92
AS14.06     MAE = 0.19     MSE = 0.03     True_Value = 7.00       Predicted = 7.19
AS14.07     MAE = 0.39     MSE = 0.15     True_Value = 5.50       Predicted = 5.89
AS14.08     MAE = 0.08     MSE = 0.01     True_Value = 6.67       Predicted = 6.75
AS14.09     MAE = 1.14     MSE = 1.30     True_Value = 6.00       Predicted = 7.14
AS14.12     MAE = 0.62     MSE = 0.38     True_Value = 6.00       Predicted = 5.38
AS14.13     MAE = 0.23     MSE = 0.05     True_Value = 8.00       Predicted = 7.77
AS14.14     MAE = 0.13     MSE = 0.02     True_Value = 7.00       Predicted = 6.87
AS14.15     MAE = 0.05     MSE = 0.00     True_Value = 7.00       Predicted = 6.95
AS14

In [8]:
absolute = []
squared = []

for participant in prediction_dict:
    true_value = prediction_dict[participant]['true_labels']
    prediction = prediction_dict[participant]['predictions']
    
    MAE = mean_absolute_error(true_value, prediction)
    MSE = mean_squared_error(true_value, prediction)
    
    absolute.append(MAE)
    squared.append(MSE)    

## ARIMA for multiple days prediction

In [43]:
X_train = pd.read_csv('data/X_train.csv')

X_test = pd.read_csv('data/X_test.csv')

y_train = pd.read_csv('data/y_train.csv')

y_test = pd.read_csv('data/y_test.csv')

splits = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}

for frame in splits:
    # We keep the id, timestamp, and value for mood
    
    try:
        splits[frame] = splits[frame][['id', 'mood']]
        splits[frame] = splits[frame].dropna()
    except:
        pass

In [46]:
y_train

,Unnamed: 0,0
0,0,6.333333
1,1,6.200000
2,2,6.400000
3,3,6.800000
4,4,6.000000
...,...,...
851,1223,6.833333
852,1224,6.000000
853,1225,7.000000
854,1226,6.000000
